In [1]:
import polars as pl
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio

# Set the theme once
pio.templates.default = "plotly_white"

from util import multi_select_counts, get_subplot_coords
import mappings

In [16]:
df = pl.read_csv("OriginalData/Preprogram_Survey.csv")

In [3]:
# Create subplot grid: 2 rows x 2 columns
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("Age", "Education", "Gender", "Race"))

# Age Histogram (Row 1, Col 1)
fig.add_trace(
    go.Histogram(x=df["How old are you?"], name="Age"),
    row=1, col=1
)

# Education Bar Chart (Row 1, Col 2)
education_counts = multi_select_counts(df, ["#", "What is your current education status?"], mappings.demograpics_map)
fig.add_trace(
    go.Bar(x=education_counts.index, y=education_counts.values, name="Education"),
    row=1, col=2
)

# Gender Bar Chart (Row 2, Col 1)
gender_counts = multi_select_counts(df, ["#", "What is your gender identity?"])
fig.add_trace(
    go.Bar(x=gender_counts.index, y=gender_counts.values, name="Gender"),
    row=2, col=1
)

# Race Bar Chart (Row 2, Col 2)
race_counts = multi_select_counts(df, ["#", 'Hispanic or Latino', 'White', 'Asian or Pacific Islander', 'American Indian or Alaska Native', 'Black or African American',])
fig.add_trace(
    go.Bar(x=race_counts.index, y=race_counts.values, name="Race"),
    row=2, col=2
)

# Update layout
fig.update_layout(height=600, width=800, title_text="Demographics Overview")
fig.show()


In [4]:
# Create subplot grid: 2 rows x 2 columns

fig = make_subplots(rows=2, cols=2,
                    subplot_titles=("Previous Job Experience", "Goal of Participation", "Career Considerations", "Development Needs"))

# Previous Job Exp Bar Chart (Row 1, Col 1)
prev_job = multi_select_counts(df, mappings.prev_job_col, mappings.prev_job_map)
fig.add_trace(
    go.Bar(x=prev_job.index, y=prev_job.values, name="Previous Job Experience"),
    row=1, col=1
)

# Goal of Participation Bar Chart (Row 1, Col 2)
goal_of_participation = multi_select_counts(df, mappings.goal_of_participation, mappings.goal_participation_map)
fig.add_trace(
    go.Bar(x=goal_of_participation.index, y=goal_of_participation.values, name="Goal of Participation"),
    row=1, col=2
)

# Career Consideration Bar Chart (Row 2, Col 1)
career = multi_select_counts(df, mappings.career_consideration_col, mappings.career_consideration_map)
fig.add_trace(
    go.Bar(x=career.index, y=career.values, name="Career Considerations"),
    row=2, col=1
)

# Development Needs Bar Chart (Row 2, Col 2)
dev_needs = multi_select_counts(df, mappings.help_needed_col, mappings.help_needed_map)
fig.add_trace(
    go.Bar(x=dev_needs.index, y=dev_needs.values, name="Development Needs"),
    row=2, col=2
)

# Update layout
fig.update_layout(height=600, width=800, title_text="Career Background and Development Needs")
fig.show()


In [5]:
fig = make_subplots(rows=2, cols=4,
                    subplot_titles=list(mappings.score_card_colname_name_mapping.keys()))

sc = df.select(["#"] + mappings.score_card).to_pandas()
sc = pl.DataFrame(sc.replace(mappings.entry_standardization_mapping))

for index, s in enumerate(mappings.score_card):
    
    position = get_subplot_coords(index, 2, 4)
    
    d = multi_select_counts(sc, ["#", s])
    fig.add_trace(
        go.Bar(y=d.values, x=d.index),
        row=position[0], col=position[1])

In [6]:
# Update layout
fig.update_layout(height=600, width=1000, title_text="Baseline Self-Assessment of Personal and Professional Skills")
fig.update_layout(showlegend=False)
fig.update_xaxes(
    categoryorder='array',
    categoryarray=['Not at all', 'Slightly', 'Moderately','Very', 'Extremely'])
fig.show()

In [8]:
df_numeric = sc[mappings.skill_cols].to_pandas().replace(mappings.skill_rating_map)

# Step 3: Average across all responses per skill
avg_scores = df_numeric.mean()

# Step 4: Plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
    r=avg_scores.tolist(),
    theta=avg_scores.index.tolist(),
    fill='toself',
    name='Average Scores'
))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[1, 5],
            tickvals=[1, 5],  
            ticktext=["Low", "High"]  
        )
    ),
    title="Overview Self-Ratings Across Key Skills",
    showlegend=False
)

fig.show()

/tmp/ipykernel_21958/352457242.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`

